# Mask SkySat DEMs

In [ ]:
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os, glob
from scipy.stats import iqr

## Define paths to data files

In [ ]:
# Input file names
site_name = 'MCS'
dem_date = '20240420'
data_path = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/{dem_date}'
ortho_fn = os.path.join(data_path, f'{site_name}_{dem_date}-1_orthomosaic.tif')
dem_fn = os.path.join(data_path, f'{site_name}_{dem_date}-1_DEM.tif')
nmad_fn = os.path.join(data_path, f'{site_name}_{dem_date}-1_nmad_mos.tif')

# Masked DEM (output) file name
dem_masked_fn = dem_fn.replace('.tif', '_masked.tif')

for fn in [dem_fn, nmad_fn, ortho_fn]:
    if not os.path.exists(fn):
        print('File does not exist, check name:', fn)

## Load DEM, NMAD, and orthomosaic

In [ ]:
# Load files
ortho = rxr.open_rasterio(ortho_fn)
ortho = xr.where(ortho==ortho._FillValue, np.nan, ortho)
dem = rxr.open_rasterio(dem_fn)
dem_crs = dem.rio.crs
dem = xr.where(dem==dem._FillValue, np.nan, dem)
nmad = rxr.open_rasterio(nmad_fn)
nmad = xr.where(nmad==nmad._FillValue, np.nan, nmad)

# Plot
cbar_shrink = 0.8
fig, ax = plt.subplots(2, 2, figsize=(10,10))
ax = ax.flatten()
# orthoimage
ortho_im = ax[0].imshow(ortho.data[0], cmap='Greys_r', 
                        extent=(np.min(ortho.x.data)/1e3, np.max(ortho.x.data)/1e3, 
                                np.min(ortho.y.data)/1e3, np.max(ortho.y.data)/1e3))
fig.colorbar(ortho_im, ax=ax[0], shrink=cbar_shrink, label='Reflectance')
# DEM
ls = matplotlib.colors.LightSource(azdeg=315, altdeg=45)
dem_im = ax[1].imshow(dem.data[0], cmap='terrain',
                      extent=(np.min(dem.x.data)/1e3, np.max(dem.x.data)/1e3, 
                              np.min(dem.y.data)/1e3, np.max(dem.y.data)/1e3))
fig.colorbar(dem_im, ax=ax[1], shrink=cbar_shrink, label='Elevation [m]')
# NMAD
nmad_im = ax[2].imshow(nmad.data[0], cmap='Reds', 
                       extent=(np.min(nmad.x.data)/1e3, np.max(nmad.x.data)/1e3, 
                               np.min(nmad.y.data)/1e3, np.max(nmad.y.data)/1e3))
fig.colorbar(nmad_im, ax=ax[2], shrink=cbar_shrink, label='NMAD [m]')
ax[3].hist(np.ravel(nmad.data), bins=100)
ax[3].set_title('NMAD [m]')

fig.tight_layout()
plt.show()

## NMAD

### Test some methods for selecting a threshold value

In [ ]:
# Interpolate DEM to NMAD coordinates
dem_interp = dem.interp(x=nmad.x, y=nmad.y, method='nearest')

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("Custom", ['w','#67000d'])

# Define thresholds
thresholds = [[1.5 * iqr(np.ravel(nmad.data), nan_policy='omit'), '1.5 x IQR(NMAD)'],
              [np.nanpercentile(np.ravel(nmad.data), 95), 'P$_{95}$(NMAD)'],
              [np.nanpercentile(np.ravel(nmad.data), 99), 'P$_{99}$(NMAD)'],
              [1, 'value'],
              [5, 'value'],
              [10, 'value'],
              [20, 'value']
             ]
                   
# Iterate over thresholds
for thresh, thresh_name in thresholds:
    mask = xr.where(nmad >= thresh, 1, 0)
    dem_interp_masked = xr.where(mask == 1, np.nan, dem_interp)
    # plot
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    nmad.plot(cmap=cmap, vmin=0, vmax=thresh, ax=ax[0])
    ax[0].set_title('NMAD')
    dem_interp_masked.plot(cmap='terrain', ax=ax[1])
    ax[1].set_title('DEM')
    fig.suptitle(f"Mask NMAD >= {thresh_name} ({np.round(thresh, 2)})")
    fig.tight_layout()
    plt.show()


### Apply threshold

In [ ]:
# Apply threshold
nmad_thresh = 2
mask = xr.where(nmad >= nmad_thresh, 1, 0)
dem_interp_masked = xr.where(mask == 1, np.nan, dem_interp)

# Write CRS
dem_interp_masked = dem_interp_masked.rio.write_crs(dem_crs)

# Plot
fig, ax = plt.subplots(1, 2, figsize=(12,6))
thresh_im = nmad.plot(cmap=cmap, vmin=0, vmax=nmad_thresh, ax=ax[0])
dem_im = dem_interp_masked.plot(cmap='terrain', ax=ax[1])
for axis in ax:
    axis.set_xticks(axis.get_xticks())
    axis.set_xticklabels(np.divide(axis.get_xticks(), 1e3).astype(str))
    axis.set_yticks(axis.get_yticks())
    axis.set_yticklabels(np.divide(axis.get_yticks(), 1e3).astype(str))
ax[0].set_xlabel('Easting [km]')
ax[0].set_ylabel('Northing [km]')
ax[1].set_xlabel('Easting [km]')
ax[1].set_ylabel('')
ax[0].set_title('NMAD')
ax[1].set_title('Masked DEM')
fig.suptitle(f"Mask NMAD >= {np.round(nmad_thresh, 2)}")
fig.tight_layout()
plt.show()


## Vegetation

### Test some difference reflectance value thresholds

In [ ]:
# Interpolate DEM to orthomosaic coordinates
dem_interp_masked_interp = dem_interp_masked.interp(x=ortho.x, y=ortho.y, method='nearest')

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("Custom", ['#67000d', 'w'])

# Define thresholds
thresholds = [[5e3, 'value'],
              [8e3, 'value'],
              [10e3, 'value'],
             ]
                   
# Iterate over thresholds
for thresh, thresh_name in thresholds:
    mask = xr.where(ortho <= thresh, 1, 0)
    dem_interp_masked_interp_masked = xr.where(mask == 1, np.nan, dem_interp_masked_interp)
    # plot
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    ortho.plot(cmap=cmap, vmin=thresh, vmax=40e3, ax=ax[0])
    ax[0].set_title('Orthomosaic value')
    dem_interp_masked_interp_masked.plot(cmap='terrain', ax=ax[1])
    ax[1].set_title('DEM')
    fig.suptitle(f"Mask orthomosaic <= {thresh_name} ({np.round(thresh, 2)})")
    fig.tight_layout()
    plt.show()


### Apply threshold

In [ ]:
# Apply threshold
refl_thresh = 10e3
mask = xr.where(ortho <= refl_thresh, 1, 0)
dem_interp_masked_interp_masked = xr.where(mask == 1, np.nan, dem_interp_masked_interp)

# Write CRS
dem_interp_masked_interp_masked = dem_interp_masked_interp_masked.rio.write_crs(dem_crs)

# Plot
fig, ax = plt.subplots(1, 2, figsize=(12,6))
thresh_im = ortho.plot(cmap=cmap, vmin=refl_thresh, vmax=4e3, ax=ax[0])
dem_im = dem_interp_masked_interp_masked.plot(cmap='terrain', ax=ax[1])
for axis in ax:
    axis.set_xticks(axis.get_xticks())
    axis.set_xticklabels(np.divide(axis.get_xticks(), 1e3).astype(str))
    axis.set_yticks(axis.get_yticks())
    axis.set_yticklabels(np.divide(axis.get_yticks(), 1e3).astype(str))
ax[0].set_xlabel('Easting [km]')
ax[0].set_ylabel('Northing [km]')
ax[1].set_xlabel('Easting [km]')
ax[1].set_ylabel('')
ax[0].set_title('Orthomosaic')
ax[1].set_title('Masked DEM')
fig.suptitle(f"Mask ortho <= {np.round(refl_thresh, 2)}")
fig.tight_layout()
plt.show()


## Save masked DEM to file

In [ ]:
# Save to file
dem_masked_fn = dem_fn.replace('.tif', f'_masked-NMAD{int(nmad_thresh)}m-refl{int(refl_thresh)}.tif')
dem_interp_masked_interp_masked.rio.to_raster(dem_masked_fn)
print('Masked DEM saved to file:', dem_masked_fn)